<a href="https://colab.research.google.com/github/dhardianto281094/AnalisisTren_Hotel/blob/main/Analisis_Tren_Hotel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Analisis Pola Booking & Prediksi Pembatalan Hotel untuk Rekomendasi Strategi Bisnis**

# Import Library

In [33]:
import pandas as pd
import numpy as np

# Read Data

In [34]:
# Atur tampilan
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Baca data
url = 'https://raw.githubusercontent.com/dhardianto281094/AnalisisTren_Hotel/main/train.csv'
df_1 = pd.read_csv(url)

In [35]:
df_1.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,bookingID
0,"Crystal Cove, Barbados Barbados",0,8,2019,January,2,10,0,4,2,0.0,0,BB,ITA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,314.0,NaN,0,Transient,38.40,0,0,Check-Out,2019-01-14,1
1,"Greensboro Courtyard Greensboro, NC",1,524,2018,December,51,15,0,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.0,NaN,0,Transient,62.80,0,0,Canceled,2017-10-21,2
2,"The Westin Peachtree Plaza, Atlanta Atlanta, GA",1,175,2019,May,20,19,1,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,12.0,NaN,0,Transient,110.00,0,0,Canceled,2018-11-25,3
3,Courtyard by Marriott Aberdeen Airport Aberdee...,0,0,2018,October,43,17,1,0,1,0.0,0,BB,NaN,Corporate,Corporate,0,0,0,A,A,0,No Deposit,NaN,200.0,0,Transient,45.00,0,0,Check-Out,2018-10-18,4
4,"W New York – Union Square New York, NY",1,33,2017,September,39,26,2,3,2,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,16.0,NaN,0,Transient,57.92,0,1,Canceled,2017-08-24,5


In [36]:
# CEK STRUKTUR DATA
# ============================================
print("Dimensi data:")
print(df_1.shape)

Dimensi data:
(83293, 33)


In [37]:
print("\nNama kolom:")
print(df_1.columns.tolist())


Nama kolom:
['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status', 'reservation_status_date', 'bookingID']


In [38]:
# Cek Tipe Data
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83293 entries, 0 to 83292
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           83293 non-null  object 
 1   is_canceled                     83293 non-null  int64  
 2   lead_time                       83293 non-null  int64  
 3   arrival_date_year               83293 non-null  int64  
 4   arrival_date_month              83293 non-null  object 
 5   arrival_date_week_number        83293 non-null  int64  
 6   arrival_date_day_of_month       83293 non-null  int64  
 7   stays_in_weekend_nights         83293 non-null  int64  
 8   stays_in_week_nights            83293 non-null  int64  
 9   adults                          83293 non-null  int64  
 10  children                        83290 non-null  float64
 11  babies                          83293 non-null  int64  
 12  meal                            

Dataset terdiri dari 83.293 baris dan 33 kolom. Tipe data bervariasi: 17 kolom numerik (int), 4 kolom desimal (float), dan 12 kolom kategori (object).

Ada beberapa kolom dengan missing values:

children (3 baris kosong)

country (346 baris kosong)

agent (11.404 baris kosong)

company (78.559 baris kosong — kemungkinan besar banyak yang tidak dipesan lewat perusahaan)

Kolom bookingID bisa digunakan sebagai identifier unik.

# Data Cleaning dan Data Manipulatif

In [39]:
# Cek missing values
print("\nMissing Values:")
print(df_1.isnull().sum())


Missing Values:
hotel                                 0
is_canceled                           0
lead_time                             0
arrival_date_year                     0
arrival_date_month                    0
arrival_date_week_number              0
arrival_date_day_of_month             0
stays_in_weekend_nights               0
stays_in_week_nights                  0
adults                                0
children                              3
babies                                0
meal                                  0
country                             346
market_segment                        0
distribution_channel                  0
is_repeated_guest                     0
previous_cancellations                0
previous_bookings_not_canceled        0
reserved_room_type                    0
assigned_room_type                    0
booking_changes                       0
deposit_type                          0
agent                             11404
company                

In [40]:
len(df_1)

83293

In [41]:
duplicated_rows = df_1[df_1.duplicated()]
print("Jumlah baris duplikat:", len(duplicated_rows))

Jumlah baris duplikat: 0


In [42]:
# Mengecek persentase missing value
missing_percent = df_1.isnull().mean() * 100

# Menggabungkan dengan jumlah missing-nya
missing_data = pd.DataFrame({
    'Missing Values': df_1.isnull().sum(),
    'Percentage (%)': missing_percent
}).sort_values(by='Percentage (%)', ascending=False)

# Menampilkan hanya kolom yang memiliki missing values
missing_data = missing_data[missing_data['Missing Values'] > 0]
print(missing_data)


          Missing Values  Percentage (%)
company            78559       94.316449
agent              11404       13.691427
country              346        0.415401
children               3        0.003602


Kolom company (Missing: 94.3%)
Drop kolom ini

Alasan:

Missing value terlalu besar → hampir tidak informatif.

Hanya tersedia pada 5.6% data → tidak bisa digeneralisasi dalam model machine learning.

Untuk visualisasi dan model, kolom ini justru bisa jadi noise.

In [43]:
# Drop kolom company
df_1.drop(columns=['company'], inplace=True)

# Imputasi agent dengan 0
df_1['agent'].fillna(0, inplace=True)

# Imputasi country dengan modus
df_1['country'].fillna(df_1['country'].mode()[0], inplace=True)

# Imputasi children dengan 0
df_1['children'].fillna(0, inplace=True)


<ipython-input-43-9a996aed3fff>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_1['agent'].fillna(0, inplace=True)
<ipython-input-43-9a996aed3fff>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

In [44]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83293 entries, 0 to 83292
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           83293 non-null  object 
 1   is_canceled                     83293 non-null  int64  
 2   lead_time                       83293 non-null  int64  
 3   arrival_date_year               83293 non-null  int64  
 4   arrival_date_month              83293 non-null  object 
 5   arrival_date_week_number        83293 non-null  int64  
 6   arrival_date_day_of_month       83293 non-null  int64  
 7   stays_in_weekend_nights         83293 non-null  int64  
 8   stays_in_week_nights            83293 non-null  int64  
 9   adults                          83293 non-null  int64  
 10  children                        83293 non-null  float64
 11  babies                          83293 non-null  int64  
 12  meal                            

In [45]:
# data tipe time

df_1['reservation_status_date'] = pd.to_datetime(df_1['reservation_status_date'])


arrival_date_month (tipe: object)
Makna: Nama bulan kedatangan (contoh: 'July', 'August', dll).
Saat ini berupa teks (object), padahal seharusnya bisa diubah menjadi urutan numerik agar bisa digunakan untuk:

- Visualisasi tren bulanan

- Model machine learning (setelah di-encode)

# Data Feature Engineering

In [50]:
# Mapping nama bulan ke angka
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Buat kolom tanggal dengan pengecekan kesalahan
df_1['arrival_date'] = pd.to_datetime(
    dict(
        year=df_1['arrival_date_year'],
        month=df_1['arrival_date_month'].map(month_map),
        day=df_1['arrival_date_day_of_month']
    ),
    errors='coerce'  # Invalid dates menjadi NaT
)


In [51]:
# Nama hari (Senin - Minggu)
df_1['arrival_day_of_week'] = df_1['arrival_date'].dt.day_name()

# Kuartal
df_1['arrival_quarter'] = df_1['arrival_date'].dt.quarter

# Flag akhir pekan
df_1['is_weekend_arrival'] = df_1['arrival_day_of_week'].isin(['Saturday', 'Sunday']).astype(int)


In [52]:
# Cek baris dengan tanggal tidak valid
invalid_dates = df_1[df_1['arrival_date'].isna()]
print(invalid_dates[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']])


       arrival_date_year arrival_date_month  arrival_date_day_of_month
1565                2018           February                         29
3069                2018           February                         29
3081                2018           February                         29
3792                2018           February                         29
4488                2018           February                         29
...                  ...                ...                        ...
77857               2018           February                         29
78926               2018           February                         29
79306               2018           February                         29
81571               2018           February                         29
82232               2018           February                         29

[65 rows x 3 columns]


In [49]:
# Kolom numerik (tipe int atau float)
numerical_cols = df_1.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Kolom kategorikal (tipe object atau kategori)
categorical_cols = df_1.select_dtypes(include=['object']).columns.tolist()

print("🔢 Kolom Numerikal:")
print(numerical_cols)
print("\n🔤 Kolom Kategorikal:")
print(categorical_cols)


🔢 Kolom Numerikal:
['is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'bookingID', 'arrival_quarter', 'is_weekend_arrival']

🔤 Kolom Kategorikal:
['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'arrival_day_of_week']


In [53]:
df_1.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,bookingID,arrival_date,arrival_day_of_week,arrival_quarter,is_weekend_arrival
0,"Crystal Cove, Barbados Barbados",0,8,2019,January,2,10,0,4,2,0.0,0,BB,ITA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,314.0,0,Transient,38.40,0,0,Check-Out,2019-01-14,1,2019-01-10,Thursday,1.0,0
1,"Greensboro Courtyard Greensboro, NC",1,524,2018,December,51,15,0,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.0,0,Transient,62.80,0,0,Canceled,2017-10-21,2,2018-12-15,Saturday,4.0,1
2,"The Westin Peachtree Plaza, Atlanta Atlanta, GA",1,175,2019,May,20,19,1,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,12.0,0,Transient,110.00,0,0,Canceled,2018-11-25,3,2019-05-19,Sunday,2.0,1
3,Courtyard by Marriott Aberdeen Airport Aberdee...,0,0,2018,October,43,17,1,0,1,0.0,0,BB,PRT,Corporate,Corporate,0,0,0,A,A,0,No Deposit,0.0,0,Transient,45.00,0,0,Check-Out,2018-10-18,4,2018-10-17,Wednesday,4.0,0
4,"W New York – Union Square New York, NY",1,33,2017,September,39,26,2,3,2,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,16.0,0,Transient,57.92,0,1,Canceled,2017-08-24,5,2017-09-26,Tuesday,3.0,0


In [54]:
df_1.info(

)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83293 entries, 0 to 83292
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           83293 non-null  object        
 1   is_canceled                     83293 non-null  int64         
 2   lead_time                       83293 non-null  int64         
 3   arrival_date_year               83293 non-null  int64         
 4   arrival_date_month              83293 non-null  object        
 5   arrival_date_week_number        83293 non-null  int64         
 6   arrival_date_day_of_month       83293 non-null  int64         
 7   stays_in_weekend_nights         83293 non-null  int64         
 8   stays_in_week_nights            83293 non-null  int64         
 9   adults                          83293 non-null  int64         
 10  children                        83293 non-null  float64       
 11  ba

In [55]:
import calendar

# 1. Total jumlah tamu
df_1['total_guests'] = df_1['adults'] + df_1['children'] + df_1['babies']

# 2. Total malam menginap
df_1['total_nights'] = df_1['stays_in_weekend_nights'] + df_1['stays_in_week_nights']

# 3. Apakah tamu membawa keluarga (anak atau bayi)
df_1['is_family'] = ((df_1['children'] > 0) | (df_1['babies'] > 0)).astype(int)

# 4. Rata-rata harga per orang
df_1['adr_per_person'] = df_1['adr'] / df_1['total_guests'].replace(0, 1)

# 5. Ada permintaan khusus atau tidak
df_1['has_special_request'] = (df_1['total_of_special_requests'] > 0).astype(int)

# 6. Konversi nama bulan jadi angka
month_map = {month: index for index, month in enumerate(calendar.month_name) if month}
df_1['booking_month_number'] = df_1['arrival_date_month'].map(month_map)

# 7. Pernah membatalkan sebelumnya
df_1['cancellation_history_flag'] = (df_1['previous_cancellations'] > 0).astype(int)

# 8. Room mismatch flag
df_1['room_mismatch_flag'] = (df_1['assigned_room_type'] != df_1['reserved_room_type']).astype(int)

# 9. Booking diubah setelah dibuat
df_1['booking_modified_flag'] = (df_1['booking_changes'] > 0).astype(int)

# 10. Booking berasal dari waiting list
df_1['was_waiting_list'] = (df_1['days_in_waiting_list'] > 0).astype(int)
